In [158]:
import pandas as pd
import time
import re
from crontab import CronTab
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

url = 'https://bet.hkjc.com/football/odds/odds_inplay.aspx?lang=en'
year = '2020'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(7) # let the potato rest for a while
games_info = driver.find_element_by_id('ActiveMatchesOdds')
soup = BeautifulSoup(games_info.get_attribute('innerHTML'), 'html.parser')
driver.quit()


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


In [130]:
print(soup.prettify())

<div class="matchesOddsTable HADTable ActiveMatches" data-reactroot="">
 <div>
  <div class="oHeader">
   <div class="tblHeader">
    <div class="normalheader">
     <div class="left">
      <span class="cDelim">
       <img alt="" src="/info/include/images/stroke_yellow.gif" title=""/>
      </span>
      <span>
       <label class="btext">
        In Play Betting
       </label>
       <!-- react-text: 11 -->
       <!-- /react-text -->
       <a>
        <span class="spicon sphelp">
         <img class="pointer icon" src="/football/info/images/icon_help.gif?CV=L3.05R1g_1" title="Betting Guide"/>
        </span>
       </a>
       <span>
        <!-- react-text: 16 -->
        <!-- /react-text -->
        <!-- react-text: 17 -->
        <!-- /react-text -->
       </span>
       <span style="display: inline;">
        <!-- react-text: 19 -->
        <!-- /react-text -->
        <!-- react-text: 20 -->
        <!-- /react-text -->
        <!-- react-text: 21 -->
        <!-- /react-te

In [171]:
event = [link.get("id")[2:] for link in soup.find_all("span", "nolnk span_vs")]
league = [link.img.get("title") for link in soup.find_all("div","cflag")][1:]
home = [link.text.strip() for link in soup.find_all("span", "teamname")][::2]
away = [link.text.strip() for link in soup.find_all("span", "teamname")][1::2]
time = [link.text.strip() for link in soup.find_all("div", "cesst")][1:]
codes = [y.get("id")[3:] for y in [x.span for x in soup.find_all("div", "cesst")] if y is not None]
game_url = ['https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid='+code for code in codes]

games = pd.DataFrame({'event': event, 
                      'league': league,
                      'home': home, 
                      'away': away, 
                      'time': time, 
                      'game_url': game_url})
# select only non-start games
games = games[games['time'].str.contains("Expected In Play start selling time")]
games['time'] = games['time'].apply(
    lambda x: re.sub('Expected In Play start selling time: ', '', x))

games['time'] = games.time.apply(lambda x: datetime.strptime(
current_year+'/'+x, '%Y/%d/%m %H:%M'))
games['month'] = games['time'].apply(lambda x: str(x.month))
games['day'] = games['time'].apply(lambda x: str(x.day))
games['hour'] = games['time'].apply(lambda x: str(x.hour))
games['minute'] = games['time'].apply(lambda x: str(x.minute))
games['crontab_cmd'] = games['minute']+' '+games['hour']+' '+games['day']+' '+games['month']+' * /opt/anaconda3/bin/python '+'/Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py'
# games['crontab_cmd'] = games[]
# games['crontab_cmd']
games

,event,league,home,away,time,game_url,month,day,hour,minute,crontab_cmd
4,20200215SAT46,French Division 1,Amiens,Paris Saint Germain,2020-02-16 00:30:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,0,30,30 0 16 2 * /opt/anaconda3/bin/python /Users/T...
5,20200215SAT48,Eng Premier,Norwich,Liverpool,2020-02-16 01:30:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,1,30,30 1 16 2 * /opt/anaconda3/bin/python /Users/T...
6,20200215SAT54,French Division 1,Bordeaux,Dijon,2020-02-16 03:00:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,3,0,0 3 16 2 * /opt/anaconda3/bin/python /Users/Ty...
7,20200215SAT60,Italian Division 1,Atalanta,Roma,2020-02-16 03:45:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,3,45,45 3 16 2 * /opt/anaconda3/bin/python /Users/T...
8,20200215SAT71,Argentine Division 1,Talleres,San Lorenzo,2020-02-16 08:50:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,8,50,50 8 16 2 * /opt/anaconda3/bin/python /Users/T...
9,20200216SUN6,Australian Division 1,Western Utd,Brisbane Roar,2020-02-16 13:00:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,13,0,0 13 16 2 * /opt/anaconda3/bin/python /Users/T...
10,20200216SUN10,Spanish Division 1,Sevilla,Espanyol,2020-02-16 19:00:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,19,0,0 19 16 2 * /opt/anaconda3/bin/python /Users/T...
11,20200216SUN12,Italian Division 1,Udinese,Verona,2020-02-16 19:30:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,19,30,30 19 16 2 * /opt/anaconda3/bin/python /Users/...
12,20200216SUN17,Spanish Division 1,Leganes,Betis,2020-02-16 21:00:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,21,0,0 21 16 2 * /opt/anaconda3/bin/python /Users/T...
13,20200216SUN22,Eng Premier,Aston Villa,Tottenham,2020-02-16 22:00:00,https://bet.hkjc.com/football/odds/odds_inplay...,2,16,22,0,0 22 16 2 * /opt/anaconda3/bin/python /Users/T...


In [99]:
cron = CronTab(user=True)

### get link to event

In [155]:
codes = [y.get("id")[3:] for y in [x.span for x in soup.find_all("div", "cesst")] if y is not None]
['https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid='+code for code in codes]

['https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=c64d242b-6c9a-4ec2-9efa-921f208496de',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=6f9f2b12-ab01-454c-8f30-d65bce045c2c',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=7c5489c0-507d-4d8d-99d2-43d41d318fb2',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=21e07c4c-b0a2-49ea-bc87-e9521b702b84',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=d6285b1e-805c-4edb-a072-445db7288d6f',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=83a6f5dc-e814-4077-9974-bd4fc994eacb',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=c38a4e4c-c71a-45d9-87fd-73c3c0b4f731',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=a80c455f-c8c2-4b0d-afed-4b4df123735e',
 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=06ff20ae-e1e5

### get league

In [166]:
[x.img.get("title") for x in soup.find_all("div","cflag")]


['Leagues and Tournaments',
 'Italian Division 1',
 'German Division 1',
 'German Division 1',
 'Spanish Division 1',
 'French Division 1',
 'Eng Premier',
 'French Division 1',
 'Italian Division 1',
 'Argentine Division 1',
 'Australian Division 1',
 'Spanish Division 1',
 'Italian Division 1',
 'Spanish Division 1',
 'Eng Premier',
 'Italian Division 1',
 'German Division 1',
 'Spanish Division 1',
 'Eng Premier',
 'German Division 1',
 'Italian Division 1',
 'Spanish Division 1',
 'Argentine Division 1',
 'Italian Division 1',
 'Eng Premier',
 'Argentine Division 1',
 'Argentine Division 1']

### get event id

In [169]:
[link.get("id")[2:] for link in soup.find_all("span", "nolnk span_vs")]

['20200215SAT9',
 '20200215SAT12',
 '20200215SAT13',
 '20200215SAT17',
 '20200215SAT46',
 '20200215SAT48',
 '20200215SAT54',
 '20200215SAT60',
 '20200215SAT71',
 '20200216SUN6',
 '20200216SUN10',
 '20200216SUN12',
 '20200216SUN17',
 '20200216SUN22',
 '20200216SUN23',
 '20200216SUN28',
 '20200216SUN29',
 '20200216SUN37',
 '20200216SUN40',
 '20200216SUN45',
 '20200216SUN46',
 '20200216SUN50',
 '20200217MON7',
 '20200217MON9',
 '20200217MON10',
 '20200217MON12']

In [173]:
list(games.crontab_cmd)

['30 0 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py',
 '30 1 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py',
 '0 3 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py',
 '45 3 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py',
 '50 8 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py',
 '0 13 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py',
 '0 19 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/development/data_collection/crawler.py',
 '30 19 16 2 * /opt/anaconda3/bin/python /Users/TysonWu/dev/odds-crawl-app/odds-crawl-app/d

### read schedule.csv and get info

In [194]:
arg = 'https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=83a6f5dc-e814-4077-9974-bd4fc994eacb'

schedule = pd.read_csv("data/schedule.csv")
game_info = schedule[schedule['game_url']==arg].iloc[-1,]
game_info

In [199]:
url = game_info['game_url']
event_id = game_info['event_id']
game_starting_time = game_info['time']
team_name_dict = dict(home=game_info['home'], away=game_info['away'])

In [200]:
url, event_id, game_starting_time, team_name_dict

('https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=83a6f5dc-e814-4077-9974-bd4fc994eacb',
 '20200215SAT48',
 '2020-02-16 01:30:00',
 {'home': 'Norwich', 'away': 'Liverpool'})